# Injury Analysis:

- Identify the unique injuries present in the dataset for self assessment and the diagnostic of the professional medical reviewer.
- Provide a description of the injuries.
- Analyse the relationship between self-assessment and the diagnostic of the medical reviewer.
- Decide if self assessment adds information.

In [1]:
import pandas as pd
import os
from core.constants import RICKD_PROY_RESOURCE_FOLDER, RICKD_SOURCE_DATA_FOLDER, RICKD_PROCESSED_DATA_FOLDER, RICKD_SESSION_DATA_FULL_FILE

run_data_full = pd.read_csv(RICKD_SESSION_DATA_FULL_FILE)
run_injury_data = run_data_full[['id', 'sub_id','InjDefn', 'SpecInjury', 'SpecInjury2']]
run_injury_data

,id,sub_id,InjDefn,SpecInjury,SpecInjury2
0,100433_20101005t132240,100433,training volume/intensity affected,pain,NaN
1,100434_20101117t132240,100434,training volume/intensity affected,disc degeneration,NaN
2,100537_20120703t102550,100537,2 workouts missed in a row,other,NaN
3,100560_20120717t103748,100560,no injury,NaN,NaN
4,101481_20120717t105021,101481,no injury,NaN,NaN
...,...,...,...,...,...
1827,200986_20150312t143944,200986,no injury,NaN,NaN
1828,200987_20150312t160840,200987,continuing to train in pain,pelvic malalignment,itb syndrome
1829,201100_20150409t155915,201100,training volume/intensity affected,muscle strain,ligament sprain
1830,201101_20150413t143152,201101,no injury,NaN,NaN


## Clean Injury type:
Clean injury type introfucing codes that are easier to handle. Remove duplciates and add descriptions.

In [2]:
# Combine SpecInjury and SpecInjury2 columns and get unique values
unique_combined_injuries = pd.concat([run_injury_data['SpecInjury'], run_injury_data['SpecInjury2']]).dropna().drop_duplicates()

print("### Distinct Injury Types")
for injury in sorted(unique_combined_injuries):
    print(f"- {injury}")


### Distinct Injury Types
- achilles tendonitis
- bursitis
- calf muscle strain
- chondromalacia
- compartment syndrome
- cyst
- deformity (e.g hammer/claw toes, bunion)
- deformity(ie hammer/claw toes, bunion)
- disc degeneration
- disc protrusion
- dislocation
- fibula fracture
- fibula stress fracture
- fill in specifics below
- first ray tendonitis (turf toe)
- gastrocnemius strain
- groin muscle strain
- hamstring muscle strain
- high ankle sprain with tendonosis
- hip flexor strain
- hip joint irritation
- ischial bursitis
- itb syndrome
- labral tear
- ligament sprain
- ligament tear/rupture acl
- ligament tear/rupture mcl
- ligament tear/rupture pcl
- ligament tear/rupture-acl
- low back pain
- medial tibial stress syndrome
- meniscal tear medial
- meniscal tear-lateral
- meniscal tear-medial
- metarsalgia
- metatarsal stress fracture
- muscle spasm
- muscle strain
- mva, not running related
- nerve impingement
- no injury
- non-specific injury
- osteitis pubis
- osteoarthritis

In [10]:

from core.data_quality import map_injury_codes
from core.constants import RICKD_MAP_INJURY_CODE, RICKD_MAP_INJURY_DESC, RICKD_MAP_SELF_INJURY_CODE

df = run_injury_data.copy()
df = map_injury_codes(df, RICKD_MAP_SELF_INJURY_CODE, "InjDefn", "self_injury_code")
df = map_injury_codes(df, RICKD_MAP_INJURY_CODE, "SpecInjury", "injury_code")
df = map_injury_codes(df, RICKD_MAP_INJURY_CODE, "SpecInjury2", "injury2_code")
df = map_injury_codes(df, RICKD_MAP_INJURY_DESC, "injury_code", "injury_desc")
df = map_injury_codes(df, RICKD_MAP_INJURY_DESC, "injury2_code", "injury2_desc")

clean_injury_data = df.copy()

In [11]:
# Create a new dataframe with all columns from run_data_full plus the new mapped columns from df
run_data_full_with_injury_codes = pd.concat([
    run_data_full,
    clean_injury_data[['self_injury_code', 'injury_code', 'injury2_code', 'injury_desc', 'injury2_desc']]
], axis=1)

run_data_full_with_injury_codes


,id,sub_id,datestring,filename,speed_r,age,Height,Weight,Gender,DominantLeg,...,r_peak_hip_add_velocity,l_peak_pelvic_drop_velocity,r_peak_pelvic_drop_velocity,l_vertical_oscillation,r_vertical_oscillation,self_injury_code,injury_code,injury2_code,injury_desc,injury2_desc
0,100433_20101005t132240,100433,2010-10-05 13:22:40,20101005t132240.json,1.610861,53,NaN,NaN,unknown,NaN,...,59.323001,-61.317978,-75.058744,61.125564,51.466247,volume_intensity,pain,NaN,General sensation of discomfort signaling tiss...,NaN
1,100434_20101117t132240,100434,2010-11-17 13:22:40,20101117t132240.json,2.237294,51,NaN,NaN,female,NaN,...,183.573751,-99.163244,-96.178927,76.028357,62.307667,volume_intensity,disc_dege,NaN,Breakdown and gradual loss of spinal disc cush...,NaN
2,100537_20120703t102550,100537,2012-07-03 10:25:50,20120703t102550.json,2.127441,255,173.1,67.6,female,right,...,257.430836,-62.309317,-62.804925,78.130812,81.753809,missed_2_workouts,other,NaN,Unspecified biomechanical disruption or pain w...,NaN
3,100560_20120717t103748,100560,2012-07-17 10:37:48,20120717t103748.json,2.657365,33,179.3,83.0,female,right,...,320.445059,-128.814092,-141.687354,113.867318,111.686204,no_injury,NaN,NaN,NaN,NaN
4,101481_20120717t105021,101481,2012-07-17 10:50:21,20120717t105021.json,2.625088,32,176.3,58.6,female,NaN,...,169.271213,-37.007098,-49.506248,91.595199,99.045820,no_injury,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,200986_20150312t143944,200986,2015-03-12 14:39:44,20150312t143944.json,4.876998,20,174.0,56.8,female,right,...,113.244646,-30.979746,-35.757575,89.724083,84.589713,no_injury,NaN,NaN,NaN,NaN
1828,200987_20150312t160840,200987,2015-03-12 16:08:40,20150312t160840.json,2.765022,50,164.0,60.0,female,right,...,171.902428,-74.064966,-70.457833,66.498443,68.524480,train_in_pain,pelv_mala,itb_synd,Misalignment or imbalance of pelvic bones caus...,Painful friction of iliotibial band rubbing ov...
1829,201100_20150409t155915,201100,2015-04-09 15:59:15,20150409t155915.json,2.790966,52,170.0,80.0,male,right,...,203.683223,-146.535658,-120.697186,74.753470,86.710600,volume_intensity,musc_stra,liga_spra,Muscle fibers stretched or torn due to excessi...,Stretching or tearing of ligaments due to exce...
1830,201101_20150413t143152,201101,2015-04-13 14:31:52,20150413t143152.json,2.828602,21,162.0,65.5,male,right,...,168.473505,-56.000594,-79.974041,76.066245,67.343927,no_injury,NaN,NaN,NaN,NaN


In [12]:
list(run_data_full_with_injury_codes.columns)
# Columns to delete because we replaced by the mapped values.
cols_to_delete = [
    "InjDefn",
    "SpecInjury",
    "SpecInjury2",
]

run_data_full_with_injury_codes = run_data_full_with_injury_codes.drop(columns=cols_to_delete)

run_data_full_with_injury_codes.to_csv(os.path.join(RICKD_PROCESSED_DATA_FOLDER, "run_data_session_with_injury.csv"))

## Analyse Injury data:
- Describe types of injuries.
- Relationship between self-assessment and the diagnostic of the medical reviewer.